In [2]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration
import portfolio
import dataAck
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
import multiprocessing as mp 
import autoPortfolioTree
import curveTreeDB
import portfolio

In [3]:
dataObjs = curveTreeDB.getValidModels(params.treeModels, returnEntireObject=True)

In [7]:
allModels = []
tickersSeen = []
for item in dataObjs:
    try:
        if item["IS_PROFITABILITY SLIPPAGE"] > 0.51 and item["IS_ANNUALIZED RETURN"] > 0.10:
            model = item["model"]
            print(model.targetTicker, model.getHash(), item["IS_SHARPE SLIPPAGE"], item["IS_SHARPE DIFFERENCE SLIPPAGE"], item["IS_BETA"])
            allModels.append(model)
            if model.targetTicker not in tickersSeen:
                tickersSeen.append(model.targetTicker)
    except:
        continue
        

IBB 9af2f1543c2c210fcc4f32b8a356d9e6fd3dc34f44176bbf8b0344be 0.7524735793133273 9.858682114316508e-05 0.47256109699048787
XLV a3e488238f8dd4831c31d24d96088379fe03cea93fdbb3f77680c12f 0.804832059798891 0.0004415358130608338 0.38373328551051206
XBI 391974b91c18a971aaf9d73a7311ce600124e8c1e81cb7606178c3c0 0.7320107262366184 0.0020862046629825937 0.52037935595012
XLV 3452ea1031b88a84ac00bd6dffac8f519fb3ca5474d0f1e7553b327d 0.6569288104342752 0.00288532574854794 0.38002052750536147
XBI 74d57b245c65edc16ba9543db1532ae054cbe2351c87d471d3efcb96 0.7857980316611682 0.003950267828858323 0.46332945928677816
IBB ec28e91d069349236182214044dc32d523d7463c900e57c9df5d7476 0.6874454913346916 0.004350642580905895 0.5097965781822248
IYT 24e04b9013aa07c5486e1dc8d433e1e76f1c98a547a780fb024657bf 0.8119993502473207 0.004460887515734702 0.3454585848152516
IBB 3f21aabec4b0d80a9fd58ec29bd9cfdb9e85ea1867693fcdd2bec3e0 0.7597187938492257 0.004732121853023075 0.4050689123792234
XBI de4058b26d68c97c4715d39049ae72328

In [8]:
len(allModels)

420

In [9]:
len(tickersSeen)

31

In [10]:
import random
factorToTrade = "VTI"#tickersSeen[random.randint(0, len(tickersSeen) - 1)]
factorToTrade

'VTI'

In [11]:
import importlib
importlib.reload(dataAck)
uniqueModels, modelReturns, modelPredictions, modelSlippageReturns, modelReturnsWithFactor, joinedData = autoPortfolioTree.computeReturnsForUniqueModelsCache(allModels, factorToTrade)


['IBB', 'XLV', 'XBI', 'IYT', 'XRT', 'SOXX', 'XHB', 'IYR', 'IVW', 'IWM', 'IWN', 'SMH', 'VO', 'IVE', 'XLF', 'XLY', 'IYF', 'KRE', 'QQQ', 'VGT', 'XLB', 'VNQ', 'VUG', 'IJR', 'XLI', 'VGK', 'SDY', 'SLV', 'XME', 'KBE', 'USO', 'VTI']
XRT
XBI
XHB
IBB
IYT
SOXX
XLV
IYR
IVW
IWM
IWN
SMH
KRE
VO
IYF
IVE
XLY
XLF
VGT
QQQ
VNQ
VUG
XLB
VGK
SDY
IJR
SLV
XME
XLI
USO
KBE
VTI
ATTEMPTING PULL 9af2f1543c2c210fcc4f32b8a356d9e6fd3dc34f44176bbf8b0344be
((('VWO', 11, 22, None, 1), 'IBB', 5, 10, 0.3, 0.05, 10), ((('VFH', 15, None, None, 3), 'IBB', 22, 15, 1.0, 0.05, 10), ((((('VGT', 35, None, 13, 2), 'IBB', 10, 2, 0.5, 0.05, 10), ((('XLF', 26, 13, None, 2), 'IBB', 44, 5, 1.5, 0.1, 10), (('IYT', 38, None, None, 1), 'IBB', 10, 3, 0.7, 0.01, 10), 5, 'AND'), 5, 'OR'), (((('IWD', 30, 1, None, 3), 'IBB', 5, 7, 0.3, 0.05, 10), (('XLE', 13, None, None, 4), 'IBB', 5, 2, 0.5, 0.05, 10), 7, 'OR'), ((('EWZ', 43, None, None, 3), 'IBB', 10, 5, 0.5, 0.1, 10), (('TLT', 42, 21, None, 3), 'IBB', 22, 3, 1.0, 0.05, 10), 5, 'OR'), 7, 'AND

In [71]:
cleanedReturns = modelReturns.fillna(0)
cleanedReturns.columns = [item.getHash() for item in uniqueModels]

cleanedPredictions = modelPredictions.fillna(0)
cleanedPredictions.columns = [item.getHash() for item in uniqueModels]
hashToModel = {}
for item in uniqueModels:
    hashToModel[item.getHash()] = item

In [72]:
cleanedReturns

,9af2f1543c2c210fcc4f32b8a356d9e6fd3dc34f44176bbf8b0344be,a3e488238f8dd4831c31d24d96088379fe03cea93fdbb3f77680c12f,391974b91c18a971aaf9d73a7311ce600124e8c1e81cb7606178c3c0,3452ea1031b88a84ac00bd6dffac8f519fb3ca5474d0f1e7553b327d,74d57b245c65edc16ba9543db1532ae054cbe2351c87d471d3efcb96,ec28e91d069349236182214044dc32d523d7463c900e57c9df5d7476,24e04b9013aa07c5486e1dc8d433e1e76f1c98a547a780fb024657bf,3f21aabec4b0d80a9fd58ec29bd9cfdb9e85ea1867693fcdd2bec3e0,de4058b26d68c97c4715d39049ae723280e0aa72605638c391a1e212,2b60b63bb3279f50a8305578ca3f921977e1089a667ee338793558a2,...,47a592ff3dd20078499ffae02c0a1ffe82dae98f5e2726aa6c4c8d12,4219d55554e23888ac87cc85811e617d32834a5ba3b5e5169641fdd7,c4b1e235ba4b35a5f876b8542fa46b43f081f8a65cc34ebfd072ee4b,d7e78ad693ecaa48ee74723f2ade6d0763ca33fcc7c3f45d2e30ec7b,67dbf302ca321a32a15d5d50824fabad4666b0125a7318742f4790dc,f6c933e3f14cafcf1d62ce96600992acefc2524a275f10089347dcdf,01f512c73c35a259acdcf281fe107e22a7c960ae9f4b7d274125d95b,93f9ae45b7d54869392c1e3c652c2baeb1a13539e63160bdbc367511,80b817df24fceb56eb4fc51e167b17beb5287ac4658b14c76c8664dd,dfda05678d76d5e86cfe74b7816f54f7cafcfe3794318080a66b2877
2008-07-15,0.000406,0.007228,0.000000,0.007228,0.000000,0.000813,0.000000,-0.000406,0.000000,-0.002845,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-16,0.000000,0.009688,0.000000,0.009688,0.000000,0.002812,0.000000,-0.001406,0.000000,-0.004218,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-17,0.000832,-0.000355,0.000000,-0.000355,0.000000,-0.004989,0.000000,0.002495,0.000000,0.004989,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-18,-0.001474,-0.017419,0.000000,-0.017419,0.000000,0.005894,0.000000,-0.002210,0.000000,-0.004421,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-21,-0.004444,0.013748,0.000000,0.013748,0.000000,0.011852,0.000000,-0.004444,0.000000,-0.008889,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-22,-0.000627,0.004283,0.000000,0.004283,0.000000,0.001253,0.000000,-0.000470,0.000000,-0.001097,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-23,-0.000188,-0.001066,0.000000,-0.001066,0.000000,0.000375,0.000000,-0.000094,0.000000,-0.000281,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-24,-0.001718,0.006048,0.000000,0.006048,0.000000,0.003436,0.000000,-0.000430,0.000000,-0.002148,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-25,-0.001901,-0.007591,0.000000,-0.007591,0.000000,0.002281,0.000000,0.000000,0.000000,-0.001901,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-07-28,-0.001204,0.008960,0.000000,0.008960,0.000000,0.001204,0.000000,-0.000301,0.000000,-0.001204,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [73]:
def historicalWeightsToTickerAllocations(historicalWeights, algorithmPredictions, modelsInPortfolio):
    aggregatePredictions = algorithmPredictions.dropna()
    allocationsToStore = []
    historicalAllocations = None
    scaledHistoricalAllocations = None
    ##ITERATE THROUGH DAYS TO CALCULATE NET POSITION
    for i in range(len(historicalWeights)):
        netPosition = {}
        weights = historicalWeights.iloc[i]
        for model in modelsInPortfolio:
            if model.targetTicker not in netPosition:
                netPosition[model.targetTicker] = 0.0
            try:
                aggregatePredictions.loc[historicalWeights.index[i]]
            except:
                continue
            
            netPosition[model.targetTicker] += weights[model.getHash()] * aggregatePredictions.loc[historicalWeights.index[i]][model.getHash()]
        thisDf = pd.DataFrame([netPosition], index=[historicalWeights.index[i]])
        if historicalAllocations is None:
            historicalAllocations = thisDf
        else:
            historicalAllocations = pd.concat([historicalAllocations, thisDf])
        
        totalCapitalUsed = sum([abs(netPosition[ticker]) for ticker in netPosition])
        scaledNetPosition = {}
        for ticker in netPosition:
            scaledNetPosition[ticker] = netPosition[ticker] * 1.0/totalCapitalUsed
        
        thisDf = pd.DataFrame([scaledNetPosition], index=[historicalWeights.index[i]])
        if scaledHistoricalAllocations is None:
            scaledHistoricalAllocations = thisDf
        else:
            scaledHistoricalAllocations = pd.concat([scaledHistoricalAllocations, thisDf])
    
    return historicalAllocations, scaledHistoricalAllocations
            
            

In [95]:
import empyrical
def getLimitedDataForPortfolio(historicalWeights, historicalPredictions, modelsUsed, factorToTrade, joinedData):
    
    normalTickerAllocationsTable, scaledTickerAllocationsTable = historicalWeightsToTickerAllocations(historicalWeights, historicalPredictions, modelsUsed)
    
    tickerAllocationsTable = scaledTickerAllocationsTable
    rawTickerPerformance = portfolioGeneration.calculatePerformanceForTable(tickerAllocationsTable, tickerAllocationsTable.columns, joinedData)

    rawAlgoPerformance = pd.DataFrame(rawTickerPerformance.apply(lambda x:sum(x), axis=1), columns=["Algo Return Without Commissions"])

    tickerPerformance, algoPerformance, algoTransactionCost =  portfolioGeneration.calculatePerformanceForAllocations(tickerAllocationsTable, joinedData)

    benchmark = factorToTrade
    factorReturn = dataAck.getDailyFactorReturn(benchmark, joinedData)
    factorReturn.columns = ["Factor Return (" + benchmark + ")"]
    algoPerformance.columns = ["Algo Return"]
    
    tickersUsed = []
    for mod in modelsUsed:
        tickersUsed.append(mod.targetTicker)
    
#     for ticker in tickersUsed:
#         thisFactorReturn = dataAck.getDailyFactorReturn(ticker, joinedData)
#         thisFactorReturn.columns = ["Factor Return (" + ticker + ")"]
#         alpha, beta = empyrical.alpha_beta(algoPerformance, thisFactorReturn)
#         print(ticker, beta)

    alpha, beta = empyrical.alpha_beta(algoPerformance, factorReturn)
    sharpe_difference = empyrical.sharpe_ratio(algoPerformance) - empyrical.sharpe_ratio(factorReturn)
    annualizedReturn = empyrical.annual_return(algoPerformance)[0]
    annualizedVolatility = empyrical.annual_volatility(algoPerformance)
    
    ##AUTOMATICALLY TAKES SLIPPAGE INTO ACCOUNT
    return {
        "benchmark":factorToTrade,
        "alpha":alpha,
        "beta":beta,
        "sharpe difference":sharpe_difference,
        "annualizedReturn":annualizedReturn,
        "annualizedVolatility":annualizedVolatility,
        "sharpe":empyrical.sharpe_ratio(algoPerformance),
        "free return":annualizedReturn - annualizedVolatility
    }
    


In [96]:
def returnSelectAlgos(algoColumns):
    return np.random.choice(algoColumns, size=random.randint(15, 30), replace= False)


In [97]:
import hrpPortfolioOpt as hrp
def produceHRPPredictions(aggregateReturns, windowSize, startIndex, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize
    if startIndex is not None:
        i = startIndex
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            
            cov = (aggregateReturns[i-windowSize:i]).cov()
        try:
            weights = hrp.getHRP(cov, corr)
        #     display(weights)
        #     display(aggregateReturns[i+windowSize:i+windowSize+1])
            todayReturn = aggregateReturns[i:i+1] * weights
        #     display(todayReturn)
            sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
            hrpReturns = pd.concat([hrpReturns, sumReturn])
            thisWeights = pd.DataFrame([[weights[item] for item in weights.index]], index=sumReturn.index, columns=weights.index.tolist())
            historicalWeights = pd.concat([historicalWeights, thisWeights])
        except:
            print("FAILED:",i)
        i += 1
    return hrpReturns, historicalWeights

In [98]:
def storeDiscoveredPortfolio(models, portfolioType, benchmark, IS_DATA, OOS_DATA):
    description = "AUTO GENERATED"
    seenTickers = []
    
    allHashes = []
    for model in models:
        allHashes.append(model.getHash())
        if model.targetTicker not in seenTickers:
            seenTickers.append(model.targetTicker)
        
    ##SORT SO ENSURE SAME PORTFOLIO NOT CREATED TWICE
    allHashes = sorted(allHashes)
    
    portfolioString = str(allHashes) + benchmark + description + portfolioType
    portfolioHash = hashlib.sha224(portfolioString.encode('utf-8')).hexdigest()
    print("PORTFOLIO HASH:", portfolioHash)
    for hashing in allHashes:
        print(hashing)
        
    
        ##UPLOAD ORGANISM OBJECT
        while True:
            try:
                toUpload = {
                    "portfolio":portfolioHash,
                    "model":hashing
                }
                datastoreClient = datastore.Client('money-maker-1236')
                #HASH DIGEST
                key = datastoreClient.key(params.discoveredPortfolioModels, hashlib.sha224(str(hashing + portfolioHash).encode('utf-8')).hexdigest()) #NEED TO HASH TO ENSURE UNDER COUNT
                organismToStore = datastore.Entity(key=key)
                organismToStore.update(toUpload)
                datastoreClient.put(organismToStore)
                break
            except:
                print("UPLOAD ERROR:", str(sys.exc_info()))
                time.sleep(10)
    
    ##STORE PORTFOLIO OBJECT
    while True:
        try:
            toUpload = {
                "description":description,
                "benchmark":benchmark,
                "portfolioType":portfolioType,
                "startedTrading":curveTreeDB.getToday()
            }
            
            for k in IS_DATA:
                toUpload["IS_"+ k] = IS_DATA[k]
            
            for k in OOS_DATA:
                toUpload["OOS_"+ k] = OOS_DATA[k]
                
            for ticker in seenTickers:
                toUpload[ticker] = True
            
            toUpload["TICKERS TRADED"] = len(seenTickers)
            
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            key = datastoreClient.key(params.discoveredPortfolios, portfolioHash) #NEED TO HASH TO ENSURE UNDER COUNT
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            datastoreClient.put(organismToStore)
            return portfolioHash
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [99]:
def getWeightingForAlgos(allModels, columns):
    countPerTicker = {}
    hashes = {}
    for mod in allModels:
        hashes[mod.getHash()] = mod.targetTicker
        if mod.targetTicker not in countPerTicker:
            countPerTicker[mod.targetTicker] = 0.0
        countPerTicker[mod.targetTicker] += 1.0
    weightsToSend = []
    for col in columns:
        weightsToSend.append(1.0/countPerTicker[hashes[col]])
        
    return [item/sum(weightsToSend) for item in weightsToSend]

In [100]:
def binarizeReturns(returnArr):
    newArr = []
    for item in returnArr:
        if item > 0.0:
            newArr.append(1.0)
        elif item < 0.0:
            newArr.append(-1.0)
        else:
            newArr.append(0.0)
    return newArr

In [101]:
def performPortfolioPerformanceEstimation(historicalPredictions, historicalReturns, factorToTrade, portfolioType, hashToModel, joinedData):
    returnWindows = [(0, historicalReturns[:450]), (450, historicalReturns)]
    historicalWeights = None
    for selectedReturns in returnWindows:
        startIndex = selectedReturns[0]
        returnWindow = selectedReturns[1]
        weightsSeen = None
        if portfolioType == "HRP FULL":
            hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,\
                    126, startIndex=max(startIndex, 126), maxWindowSize=False)
        elif portfolioType == "HRP BINARY":
            hrpReturns, weightsSeen = produceHRPPredictions(pd.DataFrame(returnWindow.apply(lambda x:binarizeReturns(x), axis=1)),\
                    126, startIndex=max(startIndex, 126), maxWindowSize=False)
        elif portfolioType == "HRP WINDOW":
            hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,\
                    126, startIndex=max(startIndex, 126), maxWindowSize=True)
        elif portfolioType == "EW":
            weightsSeen = pd.DataFrame(returnWindow.apply(lambda x: [1.0/len(x) for item in x], axis=1), columns=returnWindow.columns.values)
        elif portfolioType == "EW By Ticker":
            weightArray = getWeightingForAlgos(allModels, returnWindow.columns)
            weightsSeen = pd.DataFrame(returnWindow.apply(lambda x: weightArray, axis=1), columns=returnWindow.columns.values)
            
        
        
        if historicalWeights is None:
            historicalWeights = weightsSeen
        else:
            historicalWeights = pd.concat([historicalWeights, weightsSeen])
        
        
        modelsUsed = []

        tickersSeen = {}

        for modelHash in historicalPredictions.columns:
            thisModel = hashToModel[modelHash]
            modelsUsed.append(thisModel)
        if startIndex == 0:
            scaledStats = getLimitedDataForPortfolio(historicalWeights,\
                                    historicalPredictions, modelsUsed, factorToTrade, joinedData)
            print(scaledStats)
            if scaledStats["sharpe difference"] < 0.0 or scaledStats["annualizedReturn"] < scaledStats["annualizedVolatility"]:
                return None, None
    trainStats = getLimitedDataForPortfolio(historicalWeights[:-252], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    testStats = getLimitedDataForPortfolio(historicalWeights[-252:], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    
    if trainStats["sharpe difference"] > 0.0 and trainStats["annualizedReturn"] > trainStats["annualizedVolatility"]:
        print("ACCEPTED", trainStats, testStats)
#         storeDiscoveredPortfolio(modelsUsed, portfolioType, factorToTrade, trainStats, testStats)
    else:
        print("FAILED", trainStats)
    

In [102]:
types = ["EW"]#["HRP BINARY", "EW", "HRP WINDOW", "HRP FULL", "EW By Ticker"]

In [103]:
def createPossiblePortfolioSingle(cleanedPredictions, cleanedReturns, hashToModel, joinedData):
    selectedAlgorithms = returnSelectAlgos(cleanedReturns.columns)
    factorToTrade = "VTI"
    portfolioType = types[random.randint(0, len(types) - 1)]
    print(factorToTrade, len(selectedAlgorithms), portfolioType)
    performPortfolioPerformanceEstimation(cleanedPredictions[selectedAlgorithms],\
                    cleanedReturns[selectedAlgorithms], factorToTrade, portfolioType, hashToModel, joinedData)
        

In [104]:
createPossiblePortfolioSingle(cleanedPredictions, cleanedReturns, hashToModel, joinedData)

VTI 17 EW
{'benchmark': 'VTI', 'alpha': 0.3638421813366455, 'beta': -0.12890635070941045, 'sharpe difference': 1.1620223753146286, 'annualizedReturn': 0.38783848621573624, 'annualizedVolatility': 0.2172306779204506, 'sharpe': 1.6190028661397891, 'free return': 0.17060780829528566}


KeyboardInterrupt: 

In [26]:
## MP RUN  

def createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse):
    mpEngine = mp.get_context('fork')
        
    runningP = []
    while True:
        selectedAlgorithms = returnSelectAlgos(cleanedReturns.columns)
        factorToTrade = "VTI"#hashToModel[selectedAlgorithms[random.randint(0, len(selectedAlgorithms) - 1)]].targetTicker
        
        while len(runningP) > threadsToUse:
            runningP = dataAck.cycleP(runningP)
            
        portfolioType = types[random.randint(0, len(types) - 1)]
        print(factorToTrade, len(selectedAlgorithms), portfolioType)
        
        p = mpEngine.Process(target=performPortfolioPerformanceEstimation, args=(cleanedPredictions[selectedAlgorithms],\
                    cleanedReturns[selectedAlgorithms], factorToTrade, portfolioType, hashToModel, joinedData))
        p.start()
        runningP.append(p)

# In[ ]:


In [27]:
print("STARTING GENERATION")

##REMOVE BREAK TO DO FULL AUTO
createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse=0)

STARTING GENERATION
VTI 29 EW By Ticker
                 IBB       IVW       IWM       IWN       IYF       IYR  \
2009-08-27 -0.000413 -0.135208  0.210714  0.098333  0.045655  0.000000   
2009-08-28  0.011797 -0.123869  0.265433  0.082579  0.038340  0.000000   
2009-10-09 -0.002905 -0.135043 -0.092601 -0.014405  0.027780  0.033611   
2009-10-12 -0.013121 -0.169154  0.000000  0.017349  0.033459  0.034698   
2009-10-13 -0.006807 -0.195288  0.123992  0.057863  0.037198  0.035361   
2009-10-14  0.017062 -0.228418  0.145027  0.090239  0.043508  0.033840   
2009-10-15  0.039643 -0.294841  0.000000  0.151201  0.064800  0.035280   
2009-10-16  0.028088 -0.177249 -0.217040  0.118166  0.036173  0.014067   
2009-10-19  0.039199 -0.158508 -0.194092  0.135864  0.029114  0.012580   
2009-10-20  0.032348 -0.131228 -0.224963  0.116647  0.024996  0.005832   
2009-10-21  0.036886 -0.114011 -0.244309  0.114011  0.018323  0.001583   
2009-10-22  0.032577 -0.120308 -0.229159  0.106941  0.015277 -0.001485  

KeyboardInterrupt: 